# Coroutines

A coroutine is a type of subroutine which can have multiple entry and exit points. They're useful for a number of concurrency patterns.

Python introduced coroutines as "Enhanced Generators" in PEP 342 (https://www.python.org/dev/peps/pep-0342/), by introducing the "yield expression". That is by making it possible to have `yield` on the right hand side of assignment.

In [1]:
def print_if_error(error_is="error"):
    while True:
        line = yield
        if line.startswith(error_is):
            print(line)

# Sending values

When using a yield expression, the coroutine receives a value through a call to its `send` method. Like `next`, sending a value progresses to the next `yield` and suspends execution. In fact, given an instance of a generator, say `my_gen`, calling `next(my_gen)` is the same as `my_gen.send(None)`.

Also note that before a value can be sent to a coroutine, it must be initialized with a call to `next`. This can be automated with a decorator.

In [2]:
logs = [
    "[WARN]: A new user subscribed to cat facts!",
    "[INFO]: Todays fact: Adult cats only meow to communicate with humans.",
    "[ERR]: The user has unsubscribed from cat facts!",
    "[ERR]: 0 users are subscribed",
    "[WARN]: shutting down."
]

err_printer = print_if_error("[ERR]")
next(err_printer)

for log in logs:
    err_printer.send(log)

[ERR]: The user has unsubscribed from cat facts!
[ERR]: 0 users are subscribed


# Can a generator both produce and receive?

Yes, but it can make the behaviour of the code difficult to understand.

The example below is a natural use of taking in and returning values in one function. `track_largest` is a function that yields the largest value that it's seen so far. The semantics of how this works is somewhat confusing, and it would not be appropriate to use this in a `for` loop. 

The execution pauses after a value has been produced, but before a value has been recieved. This partly explains why you must first initialize a coroutine with a call to `next`.

In [40]:
def track_largest(start=0):
    largest = start
    while True:
        next_val = yield largest
        largest = next_val if next_val > largest else largest

In [42]:
tracker = track_largest(0)
print(next(tracker))
print(tracker.send(3))
print(tracker.send(2))
print(tracker.send(100))
print(tracker.send(9000))

0
3
3
100
9000
